In [1]:

// In python use: from pyspark.sql.functions import broadcast, split, lit
import org.apache.spark.sql.functions.{broadcast, split, lit}


val matchesBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/matches.csv")
val matchDetailsBucketed =  spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/match_details.csv")


spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
val bucketedDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
     match_id STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(16, match_id));
"""
spark.sql(bucketedDDL)

matchesBucketed.select(
    $"match_id", $"is_team_game", $"playlist_id", $"completion_date"
    )
    .write.mode("append")
    .partitionBy("completion_date")
  .bucketBy(16, "match_id").saveAsTable("bootcamp.matches_bucketed")


val bucketedDetailsDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
    match_id STRING,
    player_gamertag STRING,
    player_total_kills INTEGER,
    player_total_deaths INTEGER
)
USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""
spark.sql(bucketedDetailsDDL)

matchDetailsBucketed.select(
    $"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths")
    .write.mode("append")
    .bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")


Intitializing Scala interpreter ...

Spark Web UI available at http://dd03eda02c1f:4042
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1737518978446)
SparkSession available as 'spark'


org.apache.iceberg.exceptions.ServiceFailureException:  Server error: NotFoundException: Location does not exist: s3://warehouse/bootcamp/matches_bucketed/metadata/00000-202d2d8d-7f15-4131-a499-66be82ed1977.metadata.json

In [ ]:

// Disable broadcast join to force the bucket join.
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

matchesBucketed.createOrReplaceTempView("matches")
matchDetailsBucketed.createOrReplaceTempView("match_details")

spark.sql("""
    SELECT * FROM bootcamp.match_details_bucketed mdb JOIN bootcamp.matches_bucketed md 
    ON mdb.match_id = md.match_id
    AND md.completion_date = DATE('2016-01-01')
        
""").explain()


spark.sql("""
    SELECT * FROM match_details mdb JOIN matches md ON mdb.match_id = md.match_id
        
""").explain()

// spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "1000000000000")

// val broadcastFromThreshold = matches.as("m").join(matchDetails.as("md"), $"m.match_id" === $"md.match_id")
//   .select($"m.completion_date", $"md.player_gamertag",  $"md.player_total_kills")
//   .take(5)

// val explicitBroadcast = matches.as("m").join(broadcast(matchDetails).as("md"), $"m.match_id" === $"md.match_id")
//   .select($"md.*", split($"completion_date", " ").getItem(0).as("ds"))

// val bucketedValues = matchDetailsBucketed.as("mdb").join(matchesBucketed.as("mb"), $"mb.match_id" === $"mdb.match_id").explain()
// // .take(5)

// val values = matchDetailsBucketed.as("m").join(matchesBucketed.as("md"), $"m.match_id" === $"md.match_id").explain()

// explicitBroadcast.write.mode("overwrite").insertInto("match_details_bucketed")

// matches.withColumn("ds", split($"completion_date", " ").getItem(0)).write.mode("overwrite").insertInto("matches_bucketed")

// spark.sql(bucketedSQL)

